# Segmenting and Clustering Neighborhoods in Paris City

In this notebook, we use Foursquare APIs to explore the Paris neighborhoods by getting the most common venue categories. Then, we use k-means clustering to segment the different neighborhoods.

<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Acquiring the Data</a></li>
        <li><a href="#ref2">Preprocessing</a></li>
        <li><a href="#ref3">Exploring Downtown Neighborhoods</a></li>
        <li><a href="#ref4">Analyze Downtown Neighborhoods</a></li>
        <li><a href="#ref5">Cluster Downtown Neighborhoods</a></li>
        <li><a href="#ref6">Examine Clusters</a></li>
    </ol>
</div>
<br>
<hr>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#import library for web scraping
from bs4 import BeautifulSoup 

 #library used to handle url
import urllib

print('Libraries imported.')

Libraries imported.




<a id="ref1"></a>
# 1. Acquiring the Data

BeautifulSoup libray is used to scrap data from the following wiki page:  
https://fr.geneawiki.com/index.php/Liste_des_quartiers_de_Paris

In [13]:
#define link to the wiki page containing Toronto neighborhood table
wiki_page = urllib.request.urlopen("https://fr.geneawiki.com/index.php/Liste_des_quartiers_de_Paris")

#define BeautifulSoup object using the wiki page link
soup = BeautifulSoup(wiki_page) 



<a id="ref2"></a>
# 2. Preprocessing

At this step, we will define the dataframe Paris_Neighborhoods which contains three columns (PostalCode, Borough, and Neighborhood).

In [70]:
#get table rows
table_rows=soup.dl.dd.dl.dd.table.tbody.find_all('tr')
neighborhood_list=[]

for row in table_rows[1:]:
    tds = row.find_all('td')
    dds=tds[4].dl.find_all('dd')
    for dd in dds:
        neigh = [tds[1].text.strip(), tds[3].text.strip(), dd.text.strip()]
        neighborhood_list.append(neigh)
    
#build neighborhood DataFrame
Paris_Neighborhoods=pd.DataFrame(neighborhood_list,columns=['PostalCode','Borough','Neighborhood'])


Paris_Neighborhoods.head(10)

,PostalCode,Borough,Neighborhood
0,75001,Le Louvre,01 - Saint-Germain-l'Auxerrois
1,75001,Le Louvre,02 - Les Halles
2,75001,Le Louvre,03 - Le Palais-Royal
3,75001,Le Louvre,04 - La Place-Vendôme
4,75002,La Bourse,05 - Gaillon
5,75002,La Bourse,06 - Vivienne
6,75002,La Bourse,07 - Le Mail
7,75002,La Bourse,08 - Bonne-Nouvelle
8,75003,Le Temple,09 - Les Arts-et-Métiers
9,75003,Le Temple,10 - Les Enfants-Rouges


In [88]:
#replace ' / ' by ', ' in Neighborhood column
Paris_Neighborhoods['Neighborhood']=Paris_Neighborhoods.Neighborhood.str.replace('\d\d - ','' )
Paris_Neighborhoods

,PostalCode,Borough,Neighborhood
0,75001,Le Louvre,Saint-Germain-l'Auxerrois
1,75001,Le Louvre,Les Halles
2,75001,Le Louvre,Le Palais-Royal
3,75001,Le Louvre,La Place-Vendôme
4,75002,La Bourse,Gaillon
5,75002,La Bourse,Vivienne
6,75002,La Bourse,Le Mail
7,75002,La Bourse,Bonne-Nouvelle
8,75003,Le Temple,Les Arts-et-Métiers
9,75003,Le Temple,Les Enfants-Rouges


In [72]:
#dataframe shape
Paris_Neighborhoods.shape


(80, 3)

At this step, we will use **Nominatim** package to get the latitude and longitude values of each neighborhood.

In [102]:
# define geolocator
geolocator = Nominatim(user_agent="ny_explorer")

# initialize your variable to None
latitude_list=[]
longitude_list=[]
location=None

#loog through all neighborhoods
for neigh in Paris_Neighborhoods['Neighborhood']:
    location=geolocator.geocode('{}, Paris, France'.format(neigh))
    latitude_list.append(location.latitude) 
    longitude_list.append(location.longitude)

Paris_Neighborhoods['Latitude']=latitude_list
Paris_Neighborhoods['Longitude']=longitude_list

In [103]:
Paris_Neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,75001,Le Louvre,Saint-Germain-l'Auxerrois,48.860211,2.336299
1,75001,Le Louvre,Les Halles,48.862466,2.346009
2,75001,Le Louvre,Le Palais-Royal,48.863585,2.336204
3,75001,Le Louvre,La Place-Vendôme,48.867463,2.329428
4,75002,La Bourse,Gaillon,48.869135,2.332909




<a id="ref3"></a>
# 3. Exploring Paris Neighborhoods

At this step, we will explore Paris neighborhoods unsing Foursquare APIs.

#### Define Foursquare credentials and version

In [1]:
CLIENT_ID = 'SPADJOIC3UFDNMTOSEUF204GFY5X35QWSMJ0EDHWCFDAONDK' # your Foursquare ID
CLIENT_SECRET = 'MGY1RAQ43WVXFCXMFU0XKKQAOVTDGBIY1XVMVQGF1P31HUA5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SPADJOIC3UFDNMTOSEUF204GFY5X35QWSMJ0EDHWCFDAONDK
CLIENT_SECRET: MGY1RAQ43WVXFCXMFU0XKKQAOVTDGBIY1XVMVQGF1P31HUA5


Let's define a function, **getNearbyVenues**, which can get all venues of a list of neighborhoods by given their coordinates.

In [104]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Use the function **getNearbyVenues** to get Paris venues.

In [106]:
Paris_venues = getNearbyVenues(names=Paris_Neighborhoods['Neighborhood'],
                                   latitudes=Paris_Neighborhoods['Latitude'],
                                   longitudes=Paris_Neighborhoods['Longitude']
                                  )

Saint-Germain-l'Auxerrois
Les Halles
Le Palais-Royal
La Place-Vendôme
Gaillon
Vivienne
Le Mail
Bonne-Nouvelle
Les Arts-et-Métiers
Les Enfants-Rouges
Les Archives
Saint-Avoye
Saint-Merri
Saint-Gervais
L'Arsenal
Notre-Dame
Saint-Victor
Le Jardin-des-Plantes
Le Val-de-Grâce
La Sorbonne
La Monnaie
L'Odéon
Notre-Dame-des-Champs
Saint-Germain-des-Prés
Saint-Thomas-d'Aquin
Les Invalides
L'École-Militaire
Le Gros-Caillou
Les Champs-Élysées
Le Faubourg-du-Roule
La Madeleine
L'Europe
Saint-Georges
La Chaussée-d'Antin
Le Faubourg-Montmartre
Rochechouart
Saint-Vincent-de-Paul
La Porte-Saint-Denis
La Porte-Saint-Martin
L'Hôpital-Saint-Louis
La Folie-Méricourt
Saint-Ambroise
La Roquette
Sainte-Marguerite
Le Bel-Air
Picpus
Bercy
Les Quinze-Vingts
La Salpêtrière
La Gare
La Maison-Blanche
Croulebarbe
Le Montparnasse
Parc Montsouris 2
Le Petit-Montrouge
La Plaisance
Saint-Lambert
Necker
Grenelle
Javel
Auteuil
La Muette
La Porte-Dauphine
Chaillot 3
Les Ternes
La Plaine-Monceau
Batignolles
Les Épinettes
L

In [107]:
Paris_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Saint-Germain-l'Auxerrois,48.860211,2.336299,Cour Carrée du Louvre,48.860360,2.338543,Pedestrian Plaza
1,Saint-Germain-l'Auxerrois,48.860211,2.336299,Musée du Louvre,48.860847,2.336440,Art Museum
2,Saint-Germain-l'Auxerrois,48.860211,2.336299,La Vénus de Milo (Vénus de Milo),48.859943,2.337234,Exhibit
3,Saint-Germain-l'Auxerrois,48.860211,2.336299,Pont des Arts,48.858565,2.337635,Bridge
4,Saint-Germain-l'Auxerrois,48.860211,2.336299,Cour Napoléon,48.861172,2.335088,Plaza


Let's check the size of the result and find out how many unique categories can be curated from all the returned venues

In [108]:
print('Result size:',Paris_venues.shape) 
print('There are {} uniques categories.'.format(len(Paris_venues['Venue Category'].unique())))

Result size: (5169, 7)
There are 292 uniques categories.




<a id="ref4"></a>
# 4. Analyze Downtown Neighborhoods

In [109]:
# one hot encoding
Paris_onehot = pd.get_dummies(Paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Paris_onehot['Neighborhood'] = Paris_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Paris_onehot.columns[Paris_onehot.columns.to_list().index('Neighborhood')]] \
+ list(Paris_onehot.columns[:Paris_onehot.columns.to_list().index('Neighborhood')])\
+ list(Paris_onehot.columns[Paris_onehot.columns.to_list().index('Neighborhood')+1:])

Paris_onehot = Paris_onehot[fixed_columns]

Paris_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basque Restaurant,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caucasian Restaurant,Cemetery,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,EV Charging Station,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Liquor Store,Lounge,Lyonese Bouchon,Mac & Cheese Joint,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pharmacy,Photography Lab,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Resort,Restaurant,River,Road,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Syrian Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restauran

In [110]:
Paris_onehot.shape

(5169, 293)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [111]:
Paris_grouped = Paris_onehot.groupby('Neighborhood').mean().reset_index()

In [116]:
Paris_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basque Restaurant,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caucasian Restaurant,Cemetery,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,EV Charging Station,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Liquor Store,Lounge,Lyonese Bouchon,Mac & Cheese Joint,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pharmacy,Photography Lab,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Resort,Restaurant,River,Road,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Syrian Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restauran

In [114]:
Paris_grouped.shape

(80, 293)

 Let's define function which returns the most common venues. Then, we use this function to build a dataframe containing the top 10 venues for each neighborhood in Paris.

In [117]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [125]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Paris_grouped['Neighborhood']

for ind in np.arange(Paris_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amérique,French Restaurant,Bistro,Health Food Store,Bus Stop,Brewery,Café,Supermarket,Metro Station,Bed & Breakfast,Theater
1,Auteuil,French Restaurant,Italian Restaurant,Supermarket,Japanese Restaurant,Café,Bistro,Plaza,Dessert Shop,Burger Joint,Garden
2,Batignolles,French Restaurant,Bar,Italian Restaurant,Restaurant,Pizza Place,Wine Bar,Creperie,Brazilian Restaurant,Thai Restaurant,Bakery
3,Belleville,Bar,French Restaurant,Restaurant,Pizza Place,African Restaurant,Dim Sum Restaurant,Coffee Shop,Cocktail Bar,Chinese Restaurant,Café
4,Bercy,French Restaurant,Lounge,Hotel,Beer Garden,Japanese Restaurant,Music Venue,Waterfront,Bar,Café,Museum


In [148]:
neighborhoods_venues_sorted['2nd Most Common Venue'].value_counts()

Hotel                  23
French Restaurant      20
Italian Restaurant      7
Bar                     6
Japanese Restaurant     4
Bistro                  2
Asian Restaurant        2
Bakery                  2
Ice Cream Shop          1
Plaza                   1
Theater                 1
Café                    1
Supermarket             1
Pizza Place             1
Indian Restaurant       1
Lounge                  1
Sandwich Place          1
Cocktail Bar            1
Garden                  1
Restaurant              1
Wine Bar                1
Coffee Shop             1
Name: 2nd Most Common Venue, dtype: int64



<a id="ref5"></a>
# 5. Cluster Downtown Neighborhoods

Run *k*-means to cluster the neighborhood into 4 clusters.

In [126]:
# set number of clusters
kclusters = 5

Paris_grouped_clustering = Paris_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 3, 3, 4, 4, 1, 3, 3, 4])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [127]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Paris_merged = Paris_Neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Paris_merged = Paris_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Paris_merged.reset_index(drop=True, inplace=True)

Paris_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75001,Le Louvre,Saint-Germain-l'Auxerrois,48.860211,2.336299,4,French Restaurant,Hotel,Plaza,Art Museum,Historic Site,Museum,Tea Room,Cosmetics Shop,Clothing Store,Church
1,75001,Le Louvre,Les Halles,48.862466,2.346009,4,French Restaurant,Italian Restaurant,Bakery,Bar,Hotel,Wine Bar,Restaurant,Clothing Store,Japanese Restaurant,Ice Cream Shop
2,75001,Le Louvre,Le Palais-Royal,48.863585,2.336204,4,French Restaurant,Japanese Restaurant,Italian Restaurant,Hotel,Plaza,Art Museum,Historic Site,Wine Bar,Korean Restaurant,Coffee Shop
3,75001,Le Louvre,La Place-Vendôme,48.867463,2.329428,4,Hotel,French Restaurant,Japanese Restaurant,Bookstore,Boutique,Dessert Shop,Plaza,Hotel Bar,Tea Room,Chocolate Shop
4,75002,La Bourse,Gaillon,48.869135,2.332909,4,Japanese Restaurant,Hotel,French Restaurant,Korean Restaurant,Bookstore,Wine Bar,Theater,Pastry Shop,Chocolate Shop,Udon Restaurant


Finally, let's visualize the resulting clusters

In [190]:
# define Downtown address
address = 'Paris, France'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Paris_merged['Latitude'], Paris_merged['Longitude'], Paris_merged['Neighborhood'], Paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



<a id="ref6"></a>
# 6. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

### Cluster 1: Coffee Shop

In [141]:
Paris_merged.loc[Paris_merged['Cluster Labels'] == 0, Paris_merged.columns[[2] + list(range(5, Paris_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Saint-Thomas-d'Aquin,0,French Restaurant,Hotel,Italian Restaurant,Café,Art Gallery,Tailor Shop,Bakery,Cheese Shop,Chocolate Shop,Coffee Shop
27,Le Gros-Caillou,0,French Restaurant,Italian Restaurant,Hotel,Café,Bistro,Coffee Shop,Ice Cream Shop,Dessert Shop,Cheese Shop,Chinese Restaurant
35,Rochechouart,0,French Restaurant,Hotel,Bakery,Vegetarian / Vegan Restaurant,Pizza Place,Coffee Shop,Convenience Store,Bar,Gastropub,Park
36,Saint-Vincent-de-Paul,0,French Restaurant,Indian Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Wine Shop,African Restaurant,Coffee Shop,Seafood Restaurant,Bakery
56,Saint-Lambert,0,French Restaurant,Hotel,Sports Bar,Bakery,Thai Restaurant,Italian Restaurant,Bus Stop,Café,Pizza Place,Coffee Shop
58,Grenelle,0,French Restaurant,Hotel,Japanese Restaurant,Bistro,Restaurant,Park,Italian Restaurant,Library,Coffee Shop,Basque Restaurant
64,Les Ternes,0,French Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Asian Restaurant,Thai Restaurant,Bakery,Restaurant,Sushi Restaurant,Farmers Market
65,La Plaine-Monceau,0,French Restaurant,Italian Restaurant,Bakery,Hotel,Gym Pool,Moroccan Restaurant,Steakhouse,Seafood Restaurant,Supermarket,Mediterranean Restaurant


### Cluster 2: Park

In [142]:
Paris_merged.loc[Paris_merged['Cluster Labels'] == 1, Paris_merged.columns[[2] + list(range(5, Paris_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Les Invalides,1,French Restaurant,Hotel,Café,Italian Restaurant,Plaza,History Museum,Cocktail Bar,Japanese Restaurant,Historic Site,Gourmet Shop
26,L'École-Militaire,1,French Restaurant,Hotel,Café,Bar,Plaza,Historic Site,Food Court,Garden,Farmers Market,Bistro
28,Les Champs-Élysées,1,Hotel,French Restaurant,Boutique,Tailor Shop,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Asian Restaurant,Spa,Jewelry Store
29,Le Faubourg-du-Roule,1,Hotel,French Restaurant,Italian Restaurant,Jewelry Store,Asian Restaurant,Men's Store,Movie Theater,Electronics Store,Department Store,Cosmetics Shop
30,La Madeleine,1,Hotel,French Restaurant,Boutique,Men's Store,Women's Store,Hotel Bar,Tea Room,Salad Place,Shoe Store,Theater
31,L'Europe,1,Hotel,French Restaurant,Grocery Store,Thai Restaurant,Bistro,Supermarket,Bakery,Italian Restaurant,Modern European Restaurant,Bar
33,La Chaussée-d'Antin,1,Hotel,French Restaurant,Salad Place,Theater,Gym / Fitness Center,Concert Hall,Chocolate Shop,Chinese Restaurant,Bistro,Tea Room
53,Parc Montsouris 2,1,Hotel,Italian Restaurant,Sushi Restaurant,Park,Theater,Diner,French Restaurant,Indian Restaurant,Restaurant,Empanada Restaurant
54,Le Petit-Montrouge,1,Hotel,French Restaurant,Italian Restaurant,Supermarket,Bistro,Sushi Restaurant,Bakery,Plaza,Japanese Restaurant,Café
55,La Plaisance,1,Hotel,French Restaurant,Bistro,Bar,Bakery,Beer Store,Thai Restaurant,Sushi Restaurant,Korean Restaurant,Grocery Store


### Cluster 3: Airport

In [143]:
Paris_merged.loc[Paris_merged['Cluster Labels'] == 2, Paris_merged.columns[[2] + list(range(5, Paris_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Le Jardin-des-Plantes,2,Science Museum,Garden,Zoo Exhibit,Museum,Plaza,French Restaurant,Botanical Garden,Boat or Ferry,Breakfast Spot,Zoo
48,La Salpêtrière,2,French Restaurant,Sandwich Place,Indian Restaurant,Café,Gym / Fitness Center,Italian Restaurant,Russian Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Electronics Store
50,La Maison-Blanche,2,Vietnamese Restaurant,Asian Restaurant,Chinese Restaurant,Thai Restaurant,Cantonese Restaurant,Supermarket,Park,Diner,Pastry Shop,Cambodian Restaurant
60,Auteuil,2,French Restaurant,Italian Restaurant,Supermarket,Japanese Restaurant,Café,Bistro,Plaza,Dessert Shop,Burger Joint,Garden
70,La Goutte-d'Or,2,Asian Restaurant,Supermarket,Plaza,Thai Restaurant,Chinese Restaurant,Fast Food Restaurant,Gym,Theater,Bar,Bakery
71,La Chapelle,2,Indian Restaurant,Theater,French Restaurant,Supermarket,Bistro,Hotel,Bakery,Athletics & Sports,Café,Pedestrian Plaza
74,Amérique,2,French Restaurant,Bistro,Health Food Store,Bus Stop,Brewery,Café,Supermarket,Metro Station,Bed & Breakfast,Theater
77,Saint-Fargeau,2,Plaza,Bakery,Supermarket,French Restaurant,Bar,Tram Station,Café,Diner,Pizza Place,Fish Market


### Cluster 4: Grocery

In [144]:
Paris_merged.loc[Paris_merged['Cluster Labels'] == 3, Paris_merged.columns[[2] + list(range(5, Paris_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,L'Hôpital-Saint-Louis,3,French Restaurant,Bar,Coffee Shop,Asian Restaurant,Wine Bar,Italian Restaurant,Bakery,Cambodian Restaurant,Japanese Restaurant,Southern / Soul Food Restaurant
40,La Folie-Méricourt,3,French Restaurant,Bar,Restaurant,Bakery,Wine Bar,Pizza Place,Wine Shop,African Restaurant,Japanese Restaurant,Creperie
41,Saint-Ambroise,3,French Restaurant,Restaurant,Bar,Coffee Shop,Italian Restaurant,Bakery,Sandwich Place,Bistro,Hotel,Wine Bar
42,La Roquette,3,French Restaurant,Bar,Cocktail Bar,Italian Restaurant,Pastry Shop,Supermarket,Bistro,Bakery,Japanese Restaurant,Food & Drink Shop
43,Sainte-Marguerite,3,French Restaurant,Bar,Wine Bar,Cocktail Bar,Hotel,Arts & Crafts Store,Pop-Up Shop,Noodle House,Gluten-free Restaurant,Beer Store
44,Le Bel-Air,3,French Restaurant,Hotel,Japanese Restaurant,Supermarket,Bar,Food & Drink Shop,Bookstore,Park,Cheese Shop,Gastropub
59,Javel,3,French Restaurant,Pizza Place,Sushi Restaurant,Bakery,Café,Food & Drink Shop,Spanish Restaurant,Supermarket,Discount Store,Bistro
66,Batignolles,3,French Restaurant,Bar,Italian Restaurant,Restaurant,Pizza Place,Wine Bar,Creperie,Brazilian Restaurant,Thai Restaurant,Bakery
67,Les Épinettes,3,French Restaurant,Japanese Restaurant,Hotel,Restaurant,Pizza Place,Bar,Italian Restaurant,Bakery,Flower Shop,Brazilian Restaurant
68,Les Grandes-Carrières,3,French Restaurant,Bar,Hotel,Restaurant,Italian Restaurant,Bakery,Bistro,Park,Pizza Place,Creperie


### Cluster 5: Coffee Shop, Sushi, Italian food

In [145]:
Paris_merged.loc[Paris_merged['Cluster Labels'] == 4, Paris_merged.columns[[2] + list(range(5, Paris_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Saint-Germain-l'Auxerrois,4,French Restaurant,Hotel,Plaza,Art Museum,Historic Site,Museum,Tea Room,Cosmetics Shop,Clothing Store,Church
1,Les Halles,4,French Restaurant,Italian Restaurant,Bakery,Bar,Hotel,Wine Bar,Restaurant,Clothing Store,Japanese Restaurant,Ice Cream Shop
2,Le Palais-Royal,4,French Restaurant,Japanese Restaurant,Italian Restaurant,Hotel,Plaza,Art Museum,Historic Site,Wine Bar,Korean Restaurant,Coffee Shop
3,La Place-Vendôme,4,Hotel,French Restaurant,Japanese Restaurant,Bookstore,Boutique,Dessert Shop,Plaza,Hotel Bar,Tea Room,Chocolate Shop
4,Gaillon,4,Japanese Restaurant,Hotel,French Restaurant,Korean Restaurant,Bookstore,Wine Bar,Theater,Pastry Shop,Chocolate Shop,Udon Restaurant
5,Vivienne,4,French Restaurant,Hotel,Wine Bar,Japanese Restaurant,Clothing Store,Bookstore,Salad Place,Italian Restaurant,Bistro,Creperie
6,Le Mail,4,French Restaurant,Cocktail Bar,Wine Bar,Bakery,Creperie,Italian Restaurant,Hotel,Bar,Tea Room,Spa
7,Bonne-Nouvelle,4,Hotel,French Restaurant,Wine Bar,Cocktail Bar,Thai Restaurant,Bakery,Pizza Place,Salad Place,Theater,Greek Restaurant
8,Les Arts-et-Métiers,4,French Restaurant,Hotel,Chinese Restaurant,Café,Italian Restaurant,Cocktail Bar,Restaurant,Vietnamese Restaurant,Bakery,Art Gallery
9,Les Enfants-Rouges,4,Japanese Restaurant,Wine Bar,Italian Restaurant,Bakery,Sandwich Place,Vietnamese Restaurant,Gourmet Shop,Coffee Shop,Art Gallery,Burger Joint


In [149]:
Paris_geo=r'Paris-geo.json'

In [175]:
len(Paris_merged['PostalCode'].value_counts())
Paris_Arrondissements_list=[]
Numero_Arrondissement_list=[]
for n in range(21):
    if n==1:
        Paris_Arrondissements_list.append('Paris 1er Arrondissement')
        Numero_Arrondissement_list.append(n)
    if n>1:
        Paris_Arrondissements_list.append('Paris '+str(n)+'e Arrondissement')
        Numero_Arrondissement_list.append(n)
        
Paris_Arrondissements=pd.DataFrame()
Paris_Arrondissements['Arrondissement']=Paris_Arrondissements_list
Paris_Arrondissements['Number']=Numero_Arrondissement_list
Paris_Arrondissements

,Arrondissement,Number
0,Paris 1er Arrondissement,1
1,Paris 2e Arrondissement,2
2,Paris 3e Arrondissement,3
3,Paris 4e Arrondissement,4
4,Paris 5e Arrondissement,5
5,Paris 6e Arrondissement,6
6,Paris 7e Arrondissement,7
7,Paris 8e Arrondissement,8
8,Paris 9e Arrondissement,9
9,Paris 10e Arrondissement,10


In [191]:
map_clusters.choropleth(
    geo_data=Paris_geo,
    data=Paris_Arrondissements,
    columns=['Arrondissement', 'Number'],
    key_on='feature.properties.nom',
    fill_color='YlOrRd', 
    fill_opacity=0.1, 
    line_opacity=1,
)

# display map
map_clusters